In [ ]:
import mysql.connector
from datetime import datetime
from tkinter import *
from tkinter import messagebox

def conn():
    return mysql.connector.connect(
        host='localhost',
        user='root',
        password='12345',
        database='expense_tracker'
    )

root = Tk()
root.title("Expense Tracker")
root.geometry("700x550")

frame = Frame(root)
frame.pack()

output_frame = Frame(root)
output_frame.pack(pady=10)

Label(frame, text="Expense Tracker Menu", width=35, height=2, bg="lightpink", border=3.5, font=('Arial', 16, 'bold')).pack(pady=10)

def clear_output():
    for widget in output_frame.winfo_children():
        widget.destroy()
        
def add_expense():
    clear_output()
    Label(output_frame, text="Category:").pack()
    category_var = StringVar()
    category_var.set("grocery")
    OptionMenu(output_frame, category_var, "grocery", "medicines", "bills", "clothes", "others").pack()

    Label(output_frame, text="Description:").pack()
    desc_entry = Entry(output_frame, width=50)
    desc_entry.pack()

    Label(output_frame, text="Amount:").pack()
    amt_entry = Entry(output_frame, width=20)
    amt_entry.pack()

    date_today = datetime.today().strftime('%Y-%m-%d')

    def save():
        category = category_var.get()
        description = desc_entry.get()
        try:
            amount = float(amt_entry.get())
        except ValueError:
            messagebox.showerror("Error", "Please enter a valid number for amount.")
            return

        con = conn()
        cursor = con.cursor()
        cursor.execute('''INSERT INTO expense(data,amount,category,description)
                          VALUES (%s,%s,%s,%s)''',
                       (date_today, amount, category, description))
        con.commit()
        cursor.close()
        con.close()
        messagebox.showinfo("Success", "Expense added successfully.")
        desc_entry.delete(0, END)
        amt_entry.delete(0, END)

    Button(output_frame, text="Add", bg="lightgreen", command=save).pack(pady=5)

def view_all():
    clear_output()
    con = conn()
    cursor = con.cursor()
    cursor.execute("SELECT * FROM expense")
    records = cursor.fetchall()
    cursor.close()
    con.close()

    Label(output_frame, text="Date | Amount | Category | Description", font=('Arial', 10, 'bold')).pack()
    text = Text(output_frame, width=80, height=20)
    text.pack()
    for row in records:
        text.insert(END, f'{row[1]} | {row[2]} | {row[3]} | {row[4]}\n')

def summary():
    clear_output()

    Label(output_frame, text="Enter month (1-12):").pack()
    month_entry = Entry(output_frame)
    month_entry.pack()

    Label(output_frame, text="Enter year (yyyy):").pack()
    year_entry = Entry(output_frame)
    year_entry.pack()
    
def show_summary():
        try:
            month = int(month_entry.get())
            year = int(year_entry.get())
        except ValueError:
            messagebox.showerror("Error", "Invalid month or year.")
            return

        con = conn()
        cursor = con.cursor()
        sql = '''SELECT SUM(amount), category FROM expense
                 WHERE YEAR(data) = %s AND MONTH(data) = %s
                 GROUP BY category'''
        cursor.execute(sql, (year, month))
        records = cursor.fetchall()
        cursor.close()
        con.close()

        result = Text(output_frame, width=60, height=15)
        result.pack()
        result.insert(END, "Total Amount | Category\n")
        result.insert(END, "-"*40 + "\n")
        for row in records:
            result.insert(END, f"{row[0]} | {row[1]}\n")

        Button(output_frame, text="Show Summary", bg="lightblue", command=show_summary).pack(pady=5)

def delete():
    clear_output()
    Label(output_frame, text="Enter ID to delete:").pack()
    id_entry = Entry(output_frame)
    id_entry.pack()

    def perform_delete():
        try:
            rownum = int(id_entry.get())
        except ValueError:
            messagebox.showerror("Error", "Invalid ID.")
            return

        con = conn()
        cursor = con.cursor()
        cursor.execute("DELETE FROM expense WHERE id = %s", (rownum,))
        con.commit()
        cursor.close()
        con.close()
        messagebox.showinfo("Success", "Deleted successfully.")

    Button(output_frame, text="Delete", bg="red", command=perform_delete).pack(pady=5)

def update():
    
    for widget in output_frame.winfo_children():
        widget.destroy()

    Label(output_frame, text="Enter ID to update:").pack()
    id_entry = Entry(output_frame)
    id_entry.pack()
    Label(output_frame, text="Enter new amount:").pack()
    amt_entry = Entry(output_frame)
    amt_entry.pack()

    def perform_update():
        try:
            id_val = int(id_entry.get())
            amount = float(amt_entry.get())
        except ValueError:
            messagebox.showerror("Error", "Invalid input.")
            return

        con = conn()
        cursor = con.cursor()
        cursor.execute('''UPDATE expense
                          SET amount = %s
                          WHERE id = %s''', (amount, id_val))
        con.commit()

        if cursor.rowcount == 0:
            messagebox.showwarning("Warning", "No record found with this ID.")
        else:
            messagebox.showinfo("Success", "Updated successfully.")

        cursor.close()
        con.close()

    Button(output_frame, text="Update", bg="orange", command=perform_update).pack(pady=5)
  
def delete_mul():
    clear_output()
    Label(output_frame, text="Delete By: 1. Category  2. Date  3. Description Keyword").pack()
    choice_entry = Entry(output_frame)
    choice_entry.pack()

    Label(output_frame, text="Enter value:").pack()
    val_entry = Entry(output_frame)
    val_entry.pack()

    def perform_delete():
        ch = choice_entry.get()
        val = val_entry.get()

        con = conn()
        cur = con.cursor()

        if ch == '1':
            cur.execute("DELETE FROM expense WHERE category=%s", (val,))
        elif ch == '2':
            cur.execute("DELETE FROM expense WHERE data=%s", (val,))
        elif ch == '3':
            cur.execute("DELETE FROM expense WHERE description LIKE %s", (f'%{val}%',))
        else:
            messagebox.showerror("Error", "Invalid choice.")
            return

        con.commit()
        cur.close()
        con.close()
        messagebox.showinfo("Success", "Deleted matching records.")

    Button(output_frame, text="Delete", bg="red", command=perform_delete).pack(pady=5)

Button(frame, text="Add Expense", width=35, height=2, bg="lightblue", border=3.5, command=add_expense).pack(pady=5)
Button(frame, text="Update Expense", width=35, height=2, bg="lightblue", border=3.5, command=update).pack(pady=5)
Button(frame, text="View All Expenses", width=35, height=2, bg="lightblue", border=3.5, command=view_all).pack(pady=5)
Button(frame, text="Monthly Summary", width=35, height=2, bg="lightblue", border=3.5, command=summary).pack(pady=5)
Button(frame, text="Delete Expense", width=35, height=2, bg="lightblue", border=3.5, command=delete).pack(pady=5)
Button(frame, text="Delete Multiple", width=35, height=2, bg="lightblue", border=3.5, command=delete_mul).pack(pady=5)
Button(frame, text="Exit", width=35, height=2, bg="red", border=3.5, command=root.destroy).pack(pady=10)

root.mainloop()